# Back Testing

This notebook demonstrates several approaches to running back tests: 

* Single Run
* Walk Forward 
* Monte Carlo Simulation

In [ ]:
%use @http://roboquant.org/roboquant.json
Welcome()

# Setup
We'll use a data feed, strategy and metric that come out of the box with _roboquant_. 

In [ ]:
// The data feed we'll be using 
val feed = AvroFeed.sp500()

// Use the default included Exponential Moving Average Crossover strategy
val strategy = EMAStrategy()

// For this notebook we'll only monitor some key Account metrics
val metric = AccountMetric()

// Create an instance of Roboquant 
val roboquant =  Roboquant(strategy, metric)

# Single Run
The most simple back test approach is to run a single back test over all the data available in the feed.

In [ ]:
roboquant.run(feed)

## Inspect the account

Likely the first thing you want to know after the run has finished is: how does the account look like at the end of the run? 

An account in roboquant is owned by the broker and contains the following 4 types of data: 

1. the cash balances
2. the open positions
3. the orders, both open and closed orders
4. any trades made

> Please note the use of the **summary()** method in the next notebook cells. Many classes in roboquant implement this method and it provides a nice summary of the state on an object, like in this case the account, positions and orders.


In [ ]:
// get a snapshot of the current state of the account
val account = roboquant.broker.account
account.summary()

In [ ]:
// You can also get more details, for example about the assets in the portfolio
account.positions.summary()

In [ ]:
val trades = account.trades
trades[0..10].summary()

In [ ]:
account.closedOrders.takeLast(20).summary()

## Inspect metrics
The account contains the state at the end of the run. But metrics are more powerful in that they capture state during each step of the run. So they give an overview how a metric evolves over time during the run. It is the logger who is responsible for storing or logging the metric results.  

In [ ]:
var logger = roboquant.logger
logger.metricNames

In [ ]:
logger.getMetric("account.equity").summary()

## More detailed inspections

After a quick scan of the results, you might want to dive into some more details. For example how much profit or loss did the strategy make for a single asset, like in this the first asset that was traded?

In [ ]:
val apple = feed.assets.getBySymbol("AAPL")
val appleTrades = trades.filter { it.asset == apple }
val pnl = appleTrades.realizedPNL
println("symbol=${apple.symbol} pnl=$pnl")

Or how much profit did we make each year?

In [ ]:
for (timeframe in Timeframe.fromYears(2020, 2023).split(1.years)) { 
    val pnl = trades.filter { it.time in timeframe }.realizedPNL
    println("$timeframe => profit is $pnl")
}

# Walk Forward

To move from a single run to a walk forward back test is simple and requires only a few lines of extra code. We run the back test multiple times with each iteration limited to a certain timeframe. 

In [ ]:
// This reset will clear any state the logger has
roboquant.reset()

feed.split(1.years).forEach { 
    roboquant.run(feed, it)   
}

In [ ]:
val logger = roboquant.logger

// print the max equity per run
for ((key, value) in logger.getMetric("account.equity").groupBy { it.info.run }) {
        val max = value.max().value
        println("run = $key  max-equity = $max")
}

# Monte Carlo Simulation

The problem with back testing is that we don't for sure which historic period is most representative for the present. So instead of selecting a set of predefined periods, we'll draw random periods (uniform distributed) from our feed. 

In [ ]:
roboquant.reset()

// Get 50 samples, each of 250 trading days
feed.sample(250, 50).forEach {
     roboquant.run(feed, it)
}

In [ ]:
// See how our account equity did over all the runs
val equity = logger.getMetric("account.equity")
equity.summary()

# Next Steps
A great way to get insights into the performance of your strategies is by using one of the many included charts. If you want to find out more about that, check out the **charts notebook** next. To give an example how a chart can help to visualize the possible outcomes of our Monte Carlo simulation.

In [ ]:
MetricChart(equity, useTime = false)